In [25]:
import os

import geopandas as gpd
import polars as pl
from dotenv import load_dotenv
from sqlalchemy import create_engine
import nest_asyncio

load_dotenv()
nest_asyncio.apply() 
URI = os.getenv("POSTGRES")
FETCH_API = os.getenv("HBASE_FETCH_API")
SEND_API = os.getenv("HBASE_SEND_API")

In [26]:
sql = """
select *
from geometry.boundary_town
where city_name = %(city_code)s and town_name = '大安區'
"""

engine = create_engine(URI)
with engine.connect() as conn:
    gdf = gpd.read_postgis(
        sql,
        con=conn.connection,
        geom_col='geometry',
        params={'city_code': '臺北市'} # 台北市
    )

/Users/syuanbo/Documents/Project/h3-aggregation/.venv/lib/python3.12/site-packages/geopandas/io/sql.py:185: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(
/Users/syuanbo/Documents/Project/h3-aggregation/.venv/lib/python3.12/site-packages/geopandas/io/sql.py:473: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(spatial_ref_sys_sql, con)


In [60]:
from h3_toolkit import H3Toolkit
from h3_toolkit.hbase import HBaseClient
from h3_toolkit.aggregation import SumUp


toolkit = H3Toolkit()

result = (
    toolkit
    .set_hbase_client(
        HBaseClient(
            fetch_url = FETCH_API,
            send_url = SEND_API
        )
    )
    .process_from_vector(gdf, resolution=12)
    .fetch_from_hbase(
        table_name='res12_pre_data',
        column_family='overture_roads',
        column_qualifier=['primary', 'secondary', 'tertiary', 'residential']
    )
    # .set_aggregation_strategy(
    #     {
    #         'total_count': SumUp(),
    #     }
    # )
    # .process_from_h3(
    #     target_resolution=10,
    # )
    .get_result()
)

In [ ]:
result

In [61]:
result = result.cast({'primary': pl.Int64, 'secondary': pl.Int64, 'tertiary': pl.Int64, 'residential': pl.Int64})
result.head()

hex_id,primary,residential,secondary,tertiary
str,i64,i64,i64,i64
"""8c4ba0a414259ff""",20,null,null,null
"""8c4ba0a41425bff""",20,null,null,null
"""8c4ba0a4142c9ff""",8,null,null,null
"""8c4ba0a4142cbff""",19,null,null,null
"""8c4ba0a4142d1ff""",20,null,null,null


In [62]:
import mapclassify as mc
import pandas as pd
import pydeck as pdk
from matplotlib import colormaps

INITIAL_VIEW_STATE = {
    'latitude':25.0307167767897,
    'longitude':121.5359704226339,
    'zoom':13,
    'max_zoom':16,
    'pitch':0,
    'bearing':0
}

In [63]:
def _set_color(
    data:pl.DataFrame | gpd.GeoDataFrame,
    target_col:str,
) -> pd.DataFrame | gpd.GeoDataFrame:
    """set color column for data based on target_col
    """

    # change this classifier to fit your data
    classifier = mc.NaturalBreaks(data[target_col], k=5)
    cmap = colormaps.get_cmap('Oranges')

    if isinstance(data, pl.DataFrame):
        data = data.to_pandas()
    data['interval'] = classifier.find_bin(data[target_col])
    data['color'] = data['interval'].apply(lambda x: cmap(x / data['interval'].max()))
    data['color'] = data['color'].apply(lambda c: [int(255 * i) for i in c[:3]] + [255])
    data = data.drop(columns='interval')

    return data


def show_h3(
    data:pl.DataFrame,
    target_col:str,
    h3_col='hex_id'
):
    """show h3 hexagon layer
    """

    data = data.clone()
    data = _set_color(data, target_col)
    data = data.to_dict(orient='records')

    layer_h3 = pdk.Layer(
        'H3HexagonLayer',
        data,
        get_fill_color='color',
        # get_line_color=[255, 255, 255],
        # get_elevation='p_cnt',
        get_hexagon=h3_col,
        pickable=True,
        opacity=0.4,
        stroked=False,
        filled=True,
        extruded=False,
    )

    r = pdk.Deck(
        layers=[layer_h3],
        initial_view_state=pdk.ViewState(**INITIAL_VIEW_STATE),
        tooltip={"text": f"{target_col}: {{{target_col}}}"}
    )
    return r

In [68]:
show_h3(result.drop_nulls(subset='secondary'), 'secondary')

{
  "initialViewState": {
    "bearing": 0,
    "latitude": 25.0307167767897,
    "longitude": 121.5359704226339,
    "maxZoom": 16,
    "pitch": 0,
    "zoom": 13
  },
  "layers": [
    {
      "@@type": "H3HexagonLayer",
      "data": [
        {
          "color": [
            252,
            140,
            59,
            255
          ],
          "hex_id": "8c4ba0a41503bff",
          "primary": 24.0,
          "residential": NaN,
          "secondary": 19,
          "tertiary": NaN
        },
        {
          "color": [
            252,
            140,
            59,
            255
          ],
          "hex_id": "8c4ba0a4162c9ff",
          "primary": 22.0,
          "residential": NaN,
          "secondary": 22,
          "tertiary": NaN
        },
        {
          "color": [
            252,
            140,
            59,
            255
          ],
          "hex_id": "8c4ba0a4162cbff",
          "primary": 21.0,
          "residential": NaN,
          "secondary": 21,
          "tertiary": NaN
        },
        {
          "color": [
            255,
            245,
            235,
            255
          ],
          "hex_id": "8c4ba0a417523ff",
          "primary": 20.0,
          "residential": NaN,
          "secondary": 3,
          "tertiary": NaN
        },
        {
          "color": [
            215,
            71,
            1,
            255
          ],
          "hex_id": "8c4ba0a417535ff",
          "primary": 23.0,
          "residential": NaN,
          "secondary": 33,
          "tertiary": NaN
        },
        {
          "color": [
            255,
            245,
            235,
            255
          ],
          "hex_id": "8c4ba0a434e13ff",
          "primary": 34.0,
          "residential": NaN,
          "secondary": 4,
          "tertiary": NaN
        },
        {
          "color": [
            255,
            245,
            235,
            255
          ],
          "hex_id": "8c4ba0a434e19ff",
          "primary": 52.0,
          "residential": NaN,
          "secondary": 7,
          "tertiary": NaN
        },
        {
          "color": [
            255,
            245,
            235,
            255
          ],
          "hex_id": "8c4ba0a434e53ff",
          "primary": 41.0,
          "residential": NaN,
          "secondary": 9,
          "tertiary": NaN
        },
        {
          "color": [
            253,
            207,
            161,
            255
          ],
          "hex_id": "8c4ba0a434ee9ff",
          "primary": 58.0,
          "residential": NaN,
          "secondary": 14,
          "tertiary": NaN
        },
        {
          "color": [
            215,
            71,
            1,
            255
          ],
          "hex_id": "8c4ba0a43c321ff",
          "primary": 61.0,
          "residential": NaN,
          "secondary": 28,
          "tertiary": NaN
        },
        {
          "color": [
            252,
            140,
            59,
            255
          ],
          "hex_id": "8c4ba0a43c327ff",
          "primary": 56.0,
          "residential": NaN,
          "secondary": 21,
          "tertiary": NaN
        },
        {
          "color": [
            252,
            140,
            59,
            255
          ],
          "hex_id": "8c4ba0a4e9065ff",
          "primary": 22.0,
          "residential": NaN,
          "secondary": 21,
          "tertiary": NaN
        },
        {
          "color": [
            252,
            140,
            59,
            255
          ],
          "hex_id": "8c4ba0a4e9159ff",
          "primary": 22.0,
          "residential": NaN,
          "secondary": 18,
          "tertiary": NaN
        },
        {
          "color": [
            255,
            245,
            235,
            255
          ],
          "hex_id": "8c4ba0a4e9323ff",
          "primary": 21.0,
          "residential": NaN,
          "secondary": 7,
          "tertiary":